In [1]:
# Example file showing a circle moving on screen
import math
import pygame
from pygame.locals import *
import time

import nltk
from nltk import word_tokenize, pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
# nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger_eng')

from pytmx.util_pygame import load_pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def get_player_sprites(player):
    stand = ["_stand.png"]
    walk = ["_walk1.png", "_walk2.png"]
    jump = ["_up1.png"]
    action_suffix = [stand, walk, jump]
    
    player_path_prefix = ".\kenney_abstract-platformer\PNG\Players\Player "+ player + "\player" + player
    
    player_sprites = []
    for action in action_suffix:
        action_sprites = []
        for suffix in action:
            sprite_filepath = player_path_prefix + suffix
            sprite = pygame.image.load(sprite_filepath)
            sprite = pygame.transform.scale(sprite, (MAZE_CSIZE/2+5, MAZE_CSIZE/2+5))
            action_sprites.append(sprite)
        player_sprites.append(action_sprites)
    
    return player_sprites

In [3]:
COMMAND_SET = ["stand", "move left", "move right", "jump", "change color", "command is not correct"]
# my_string = "I won a lot of apples and I am happier now."

# *******************************************************
# ***** color sets *****
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREY = (200, 200, 200)
WHITE = (255, 255, 255)

GREEN_DARK = (46, 123, 82)
GREEN_LIGHT = (139, 192, 86)
RED_BRICK = (160, 71, 71)

BROWN = (108, 84, 60)

# *******************************************************
# ***** UI resources *****
background_images = [".\kenney_abstract-platformer\PNG\Backgrounds\set4_background.png",               
                     ".\kenney_abstract-platformer\PNG\Backgrounds\set4_hills.png",
                     ".\kenney_abstract-platformer\PNG\Backgrounds\set4_tiles.png"]

box_image = ".\kenney_ui\PNG\\panel_beigeLight.png"
button_image = ".\kenney_ui\PNG\\buttonLong_beige.png"
button_pressed_image = ".\kenney_ui\PNG\\buttonLong_beige_pressed.png"

BOX_W = 700
BOX_H = 56
BTN_W = 120
BTN_H = BOX_H

# *******************************************************
# ***** goal animation resources *****
red_goal_image = '.\kenney_abstract-platformer\PNG\Other\plantRed_5.png'
blue_goal_image = '.\kenney_abstract-platformer\PNG\Other\plantBlue_1.png'

# *******************************************************
# ***** sprites *****
MAZE_GRID_TILE = 16 # 16*16-grid maze
MAZE_CSIZE = 32 # each grid cell is 32*32 pixel large in the screen
tmxfile = ".\climb_up_maze.tmx"

PLAYERS = ["Grey", "Red", "Blue"]
PLAYER_ID = 0
# PLAYER_SPRITE_PATH = ".\kenney_abstract-platformer\PNG\Players"
CUR_SPRITES = get_player_sprites(PLAYERS[PLAYER_ID])

DEVIATE = 0.3

# *******************************************************
# ***** screen *****
SCREEN_W = 1280
SCREEN_H = 720
MAZE_ANCOR = (390,100)

In [4]:
# ************************* NLP utility functions ******************************

# nltk's pos_tag() function creates a list of tuples of (word, pos_tag), the pos_tags are uppercase abbreviations,
# while nltk's lemmatize() takes a single lowercase letter as the part-of-speech tag
# the create_new_pos_tags() function is a translation function, that returns a new list of tuples that can be used directly by lemmatize()

def create_new_pos_tags(pos_tags):
    new_pos_tags = []
    for word_tuple in pos_tags:
        w = word_tuple[0]
        new_pos_tags.append((w, get_new_pos_tag(word_tuple)))
    return new_pos_tags

# return the mapping result (single lowercase letter) of the pos tags
def get_new_pos_tag(word_tuple):
    tag_dict = {"J": "a", # adjective
            "N": "n", # noun
            "V": "v", # verb
            "R": "r"} # adverb
    tag = word_tuple[1][0]
    if tag in tag_dict.keys():        
        return tag_dict[tag]
    else:
        return tag.lower()
    
def create_tokens(input_text, is_lower_cased, is_clean_punctuation):
    if is_lower_cased:
        input_text = input_text.lower()
        
    if is_clean_punctuation:
        reg_tokenizer = RegexpTokenizer(r'\w+')
        tokenized_text = reg_tokenizer.tokenize(input_text)       
    else:
        tokenized_text = word_tokenize(input_text)
     
    pos_tags = pos_tag(tokenized_text)    
    new_pos_tags = create_new_pos_tags(pos_tags)

    lemmatizer = WordNetLemmatizer()
    # only lemmatize nouns, to avoid lemmatize 'left' and 'right' to unwanted root words
    tokens = [lemmatizer.lemmatize(w,t) if t=='n' else lemmatizer.lemmatize(w) for w,t in new_pos_tags]   
    
    return tokens

def compare_command(input_text):
    tokens = create_tokens(input_text, True, True)
    
    if (len(tokens)>2) | (len(tokens)==0):
        return COMMAND_SET[-1]    
    new_sentence = " ".join(tokens)
    try:
        command_id = COMMAND_SET.index(new_sentence)
        return COMMAND_SET[command_id]
    except:
        return COMMAND_SET[-1]

In [5]:
# ************************* game functions ******************************  

# *******************************************************
# ***** utility conversions *****
def coord2pixel(x_coord, y_coord, anchor=MAZE_ANCOR):
    return x_coord*MAZE_CSIZE + anchor[0], y_coord*MAZE_CSIZE + anchor[1]

def pixel2coord(x, y, anchor=MAZE_ANCOR):
    return ((x-anchor[0])//MAZE_CSIZE), ((y-anchor[1])//MAZE_CSIZE)


# *******************************************************
# ***** blitting *****
def blit_all_tiles(screen, tmxdata, anchor=MAZE_ANCOR):
    for layer in tmxdata:
        for tile in layer.tiles():
            # tile[0] .... x grid location
            # tile[1] .... y grid location
            # tile[2] .... image data for blitting
            img = pygame.transform.scale(tile[2], (MAZE_CSIZE, MAZE_CSIZE))
            x_pixel = tile[0]*MAZE_CSIZE + anchor[0]
            y_pixel = tile[1]*MAZE_CSIZE + anchor[1]
            screen.blit(img, (x_pixel, y_pixel))

def blit_player(screen, x, y, sprite=CUR_SPRITES[0][0]):
#     show_sprite = sprite
    screen.blit(sprite, (x, y))

    
# *******************************************************    
# ***** tile property *****            
def get_tile_properties(tmxdata, x, y, anchor=MAZE_ANCOR):
    world_x = x - anchor[0]
    world_y = y - anchor[1]
    tile_x = world_x // MAZE_CSIZE
    tile_y = world_y // MAZE_CSIZE
    layer = tmxdata.layers[0]
    
    try:
        properties = tmxdata.get_tile_properties(tile_x, tile_y, 0)
    except ValueError:
        properties = {'jumpable': 0, 'point': 0, 'point_color': '0', 'start': 0, 'surface': 0, 'through': 0}
    if properties is None:
        properties = {'jumpable': 0, 'point': 0, 'point_color': '0', 'start': 0, 'surface': 0, 'through': 0}
    return properties

def get_tile_properties_grid(tmxdata, x_coord, y_coord, anchor=MAZE_ANCOR):
    layer = tmxdata.layers[0]
    try:
        properties = tmxdata.get_tile_properties(x_coord, y_coord, 0)
    except ValueError:
        properties = {'jumpable': 0, 'point': 0, 'point_color': '0', 'start': 0, 'surface': 0, 'through': 0}
    if properties is None:
        properties = {'jumpable': 0, 'point': 0, 'point_color': '0', 'start': 0, 'surface': 0, 'through': 0}
    return properties


# *******************************************************
# ***** player init *****
def find_start_tile(tmxdata):
    # returns the grid-coordinates, not pixels
    for layer in tmxdata:
        for tile in layer.tiles():
            properties = tmxdata.get_tile_properties(tile[0], tile[1], 0)
            if properties['start'] == 1:
                return tile[0], tile[1]
        
def initialize_player(screen, tmxdata, anchor=MAZE_ANCOR):
    x_coord, y_coord = find_start_tile(tmxdata)
    x, y = coord2pixel(x_coord+DEVIATE, y_coord-1+DEVIATE)
    blit_player(screen, x, y)
    return x, y


# *******************************************************
# ***** player actions *****
def check_move_left(tmxdata, x_coord, y_coord, anchor=MAZE_ANCOR):
    for step in range(0,4):     
        left_cell_property = get_tile_properties_grid(tmxdata, max([x_coord-step, 0]) , y_coord)
        if left_cell_property['through']!=1:
            return False
    return True

def check_move_right(tmxdata, x_coord, y_coord, anchor=MAZE_ANCOR):
    for step in range(0,4):
        right_cell_property = get_tile_properties_grid(tmxdata, min([x_coord+step, 15]), y_coord)
        if right_cell_property['through']!=1:
            return False
    return True

def check_jump(tmxdata, x_coord, y_coord, anchor=MAZE_ANCOR):
    for step in [1,2]:
        up_cell_property = get_tile_properties_grid(tmxdata, x_coord, y_coord-step)        
        if up_cell_property['point'] == 1:
            return 2
        
        if ((up_cell_property['jumpable'] !=1) & (up_cell_property['point'] != 1)):
            return 0
    return 1
    
def check_color_match(tmxdata, x_coord, y_coord, player=PLAYER_ID, anchor=MAZE_ANCOR):
    cell_property = get_tile_properties_grid(tmxdata, x_coord , y_coord)
    player_color = PLAYERS[player].lower()
    if cell_property['point_color'] != player_color:
        return False
    return True

def move_left(screen, tmxdata, x, y, anchor=MAZE_ANCOR):
    x_coord, y_coord = pixel2coord(x, y)
    if check_move_left(tmxdata, x_coord, y_coord):
        x_coord = max([x_coord-3,1])
        x, y = coord2pixel(x_coord+DEVIATE, y_coord+DEVIATE)
        blit_player(screen, x, y)
    return x, y
        
def move_right(screen, tmxdata, x, y, anchor=MAZE_ANCOR):
    x_coord, y_coord = pixel2coord(x, y)
    if check_move_right(tmxdata, x_coord, y_coord):
        x_coord = min([x_coord+3, 14])
        x, y = coord2pixel(x_coord+DEVIATE, y_coord+DEVIATE)
        blit_player(screen, x, y)
    return x, y

def jump(screen, tmxdata, x, y, anchor=MAZE_ANCOR):
    x_coord, y_coord = pixel2coord(x, y)
    jump_end_index = check_jump(tmxdata, x_coord, y_coord)
    match jump_end_index:
        case 1:
            y_coord = y_coord-3
            x, y = coord2pixel(x_coord+DEVIATE, y_coord+DEVIATE)
            blit_player(screen, x, y)
            return x, y
        case 2:
            y_coord = max([y_coord-3,0])
            if check_color_match(tmxdata, x_coord, y_coord, PLAYER_ID):
                x, y = coord2pixel(x_coord+DEVIATE, y_coord+DEVIATE)
                blit_player(screen, x, y)
                return x, y
            else:
                return x, y 
        case _:
            return x, y

def change_color(player_id=PLAYER_ID):
    player_id = (player_id+1)%3
    return player_id 

def check_goal(screen, tmxdata, x, y, anchor=MAZE_ANCOR):
    properties = get_tile_properties(tmxdata, x, y, anchor=MAZE_ANCOR)
    is_point_tile = properties['point']
    if is_point_tile:
        return properties['point_color']
    else:
        return ""

# *******************************************************
# ***** goal animation *****        
def blit_fade_in(screen, image, pos, end_alpha=128, time=64):
    step = end_alpha/time
    alpha = image.get_alpha()
    alpha = min(end_alpha, alpha+step)
    image.set_alpha(alpha)
    screen.blit(image, pos)
    return alpha == end_alpha    

In [6]:
pygame.init()
screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
clock = pygame.time.Clock()
# clock.tick(25)
running = True
background = GREY
tmxdata = load_pygame(tmxfile)

# *******************************************************
# ***** UI *****
bg_imgs = []
for im in background_images:
    tmp = pygame.image.load(im)
    tmp = pygame.transform.scale(tmp, (SCREEN_W, SCREEN_H))
    bg_imgs.append(tmp)
    
input_box_img = pygame.image.load(box_image)
input_box_img = pygame.transform.scale(input_box_img, (BOX_W, BOX_H))
input_box_rect = input_box_img.get_rect()
input_box_rect.center = (SCREEN_W/2, SCREEN_H-50)

btn_img = pygame.image.load(button_image)
btn_img = pygame.transform.scale(btn_img, (BTN_W, BTN_H))
btn_rect = btn_img.get_rect()
btn_rect.topleft = (input_box_rect.topright[0]+20, input_box_rect.topright[1])

btn_pressed_img = pygame.image.load(button_pressed_image)
btn_pressed_img = pygame.transform.scale(btn_pressed_img, (BTN_W, BTN_H))
btn_pressed_rect = btn_pressed_img.get_rect()
btn_pressed_rect.topleft = (input_box_rect.topright[0]+23, input_box_rect.topright[1]+3)

btn_txt = "CLEAR"
font = pygame.font.SysFont("berlinsansfb", 22, bold=True)
btn_txt_img = font.render(btn_txt, True, BROWN)
btn_txt_rect = btn_txt_img.get_rect()
btn_txt_rect.center = btn_rect.center

btn_countdown = 0

# *******************************************************
# ***** goal animation *****
goal_bg = pygame.Surface(screen.get_size(), pygame.SRCALPHA)
goal_bg.set_alpha(0)
goal_bg.fill(BLACK)
# screen.blit(goal_bg,(0,0))

goal_banner = pygame.Surface((SCREEN_W, SCREEN_H/2.6), pygame.SRCALPHA)
goal_banner.set_alpha(0)
goal_banner.fill(BLACK)
goal_banner_rect = goal_banner.get_rect()
goal_banner_rect.center = (SCREEN_W/2, SCREEN_H/2)
# screen.blit(goal_banner, goal_banner_rect)

goal_red_img = pygame.image.load(red_goal_image)
goal_red_img = pygame.transform.scale(goal_red_img, (MAZE_CSIZE*1.5, MAZE_CSIZE*1.5))
goal_red_rect = goal_red_img.get_rect()
goal_red_rect.center = (SCREEN_W/2, SCREEN_H*0.42)
goal_red_img.set_alpha(0)

goal_blue_img = pygame.image.load(blue_goal_image)
goal_blue_img = pygame.transform.scale(goal_blue_img, (MAZE_CSIZE*1.5, MAZE_CSIZE*1.5))
goal_blue_rect = goal_blue_img.get_rect()
goal_blue_rect.center = (SCREEN_W/2, SCREEN_H*0.42)

goal_color = ""
goal_txt = 'You grew the %s wonder seed!'% goal_color
# font = pygame.font.SysFont("berlinsansfb", 36, bold=True)
# goal_txt_img = font.render(goal_txt, True, WHITE)
# goal_txt_img.set_alpha(0)
# goal_txt_rect = goal_txt_img.get_rect()
# goal_txt_rect.center = (SCREEN_W/2, SCREEN_H/2)

is_fade_in = False

# *******************************************************
# ***** command input *****
text = ""
font = pygame.font.SysFont("berlinsansfb", 32)
cur_color = GREEN_DARK
text_img = font.render(text, True, cur_color)
pygame.key.set_repeat(500, 100)

text_rect = text_img.get_rect()              
text_rect.center = input_box_rect.center
cursor = Rect(text_rect.topright, (3, text_rect.height))

# *******************************************************
# ***** player controls *****
PLAYER_ID = 0
CUR_SPRITES = get_player_sprites(PLAYERS[PLAYER_ID])
cur_x, cur_y = initialize_player(screen, tmxdata)
end_x, end_y = cur_x, cur_y
# cur_x, cur_y = coord2pixel(10,2)
# blit_player(screen, cur_x, cur_y)
# end_x, end_y = cur_x, cur_y
speed = 2
cur_action = "stand"
player_walk_frame = 0

# ************************************************ game loop ***************************************************
while running:      
    mouse = pygame.mouse.get_pos()    
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False
        # *********************** clear button ********************************    
        if event.type == MOUSEBUTTONDOWN: 
             if (btn_rect.left <= mouse[0] <= btn_rect.right) & (btn_rect.top <= mouse[1] <= btn_rect.bottom): 
                    text = ""
                    cur_color = GREEN_DARK
                    btn_countdown = 1                    
                    screen.blit(btn_pressed_img, btn_pressed_rect)
                    btn_txt_rect.center = btn_pressed_rect.center
            
        # *********************** keyboard input ********************************         
        if event.type == KEYDOWN:
            if event.key == K_BACKSPACE:
                if len(text)>0:
                    text = text[:-1]                                     
            elif event.key == K_RETURN:
                cur_color = GREEN_LIGHT
                cur_action = compare_command(text)
                text = '"' + cur_action + '"'
                
                if cur_action == "move left":
                    end_x, end_y = move_left(screen, tmxdata, cur_x, cur_y,)
                    if (cur_x == end_x) & (cur_y == end_y):
                        cur_action = "stand"
                        text = "unable to move!"
                        cur_color = RED_BRICK
                    player_walk_frame = 0
                if cur_action == "move right":
                    end_x, end_y = move_right(screen, tmxdata, cur_x, cur_y,)
                    if (cur_x == end_x) & (cur_y == end_y):
                        cur_action = "stand"
                        text = "unable to move!" 
                        cur_color = RED_BRICK
                    player_walk_frame = 0                    
                if cur_action == "jump":
                    end_x, end_y = jump(screen, tmxdata, cur_x, cur_y,)
                    if (cur_x == end_x) & (cur_y == end_y):
                        cur_action = "stand"
                        text = "unable to move!" 
                        cur_color = RED_BRICK
                if cur_action == "change color":
                    PLAYER_ID = change_color(PLAYER_ID)
                    CUR_SPRITES = get_player_sprites(PLAYERS[PLAYER_ID])                    
            else:
                cur_color = GREEN_DARK
                text += event.unicode
                
    # ************************* blit background & UI ******************************
    # screen.fill(background)      
    for im in bg_imgs:        
        screen.blit(im,(0,0))

    screen.blit(input_box_img, input_box_rect)
    if 0 <= btn_countdown <= 0.1:
        btn_txt_rect.center = btn_rect.center
        screen.blit(btn_img, btn_rect)        
    else:   
        btn_countdown -= 0.5
        screen.blit(btn_pressed_img, btn_pressed_rect)        
    screen.blit(btn_txt_img, btn_txt_rect)
    
    # ************************* blit maze ******************************
    blit_all_tiles(screen, tmxdata)
    
    # ************************** blit player *****************************   
    if cur_x != end_x:
        cur_x += speed * math.copysign(1, end_x - cur_x)
    if cur_y != end_y:
        cur_y += speed*1.5 * math.copysign(1, end_y - cur_y)
                
    match cur_action:            
        case "move left":
            blit_player(screen, cur_x, cur_y, CUR_SPRITES[1][int(player_walk_frame)])            
            player_walk_frame = (player_walk_frame + 0.5) % len(CUR_SPRITES[1])
            if (abs(cur_x-end_x) <= 0.01) & (abs(cur_y-end_y) <= 0.01):
                cur_action = "stand"
                cur_x = end_x
                cur_y = end_y
        case "move right":
            blit_player(screen, cur_x, cur_y, CUR_SPRITES[1][int(player_walk_frame)])   
            player_walk_frame = (player_walk_frame + 0.5) % len(CUR_SPRITES[1])
            if (abs(cur_x-end_x) <= 0.01) & (abs(cur_y-end_y) <= 0.01):
                cur_action = "stand"
                cur_x = end_x
                cur_y = end_y
        case "jump":
            blit_player(screen, cur_x, cur_y, CUR_SPRITES[2][0])
            if (abs(cur_x-end_x) <= 0.01) & (abs(cur_y-end_y) <= 0.01):
                cur_action = "stand"
                cur_x = end_x
                cur_y = end_y
        case _:
            blit_player(screen, cur_x, cur_y, CUR_SPRITES[0][0])

            
    # ************************* blit command text ******************************      
    text_img = font.render(text, True, cur_color)    
    text_rect.size = text_img.get_size()
    text_rect.center = input_box_rect.center
    cursor.topleft = text_rect.topright
    screen.blit(text_img, text_rect)
    if time.time() % 1 > 0.5:
        pygame.draw.rect(screen, cur_color, cursor)            
            
    # ************************* blit fade-in ending scenes ******************************      
    if not is_fade_in:
        goal_color = check_goal(screen, tmxdata, cur_x, cur_y)
        if goal_color != "":
            is_fade_in = True
            goal_txt = 'You grew the %s wonder seed!'% goal_color
            font = pygame.font.SysFont("berlinsansfb", 36, bold=True)
            goal_txt_img = font.render(goal_txt, True, WHITE)
            goal_txt_img.set_alpha(0)
            goal_txt_rect = goal_txt_img.get_rect()
            goal_txt_rect.center = (SCREEN_W/2, SCREEN_H/2+20)
    else:     
        blit_fade_in(screen, goal_banner, goal_banner_rect, 200, 32)
        blit_fade_in(screen, goal_bg, (0,0), 144, 32)
        if goal_color =='red':
            blit_fade_in(screen, goal_red_img, goal_red_rect, 255, 32)
        else:
            blit_fade_in(screen, goal_blue_img, goal_blue_rect, 255, 32)
        done = blit_fade_in(screen, goal_txt_img, goal_txt_rect, 255, 32)
#         if done:
#             pygame.quit()
        
    pygame.display.update()
pygame.quit()

In [7]:
# #test if the lemmatizer transforms nouns, verbs and adjectives at once
# my_string = "I won a lot of apples and I am happier now."
# tokenized_text = word_tokenize(my_string.lower())
# pos_tags = pos_tag(tokenized_text)
# print(pos_tags)

# new_pos_tags = create_new_pos_tags(pos_tags)
# print([lemmatizer.lemmatize(w,t) if t in ['a','n','v'] else lemmatizer.lemmatize(w) 
#  for w,t in new_pos_tags])